# Task 3: Minimum Edit Distance

![MED_Cover](cover_images/MED_Cover.png)

## Part 1: Setting up the foundation

The **Minimum Edit Distance** (MED) is the minimum number of edits (insertions, deletions, substitutions) needed to transform one string into another. This is often implemented via dynamic programming (you can learn more about the algorithm in the lecture slides and notes).

### Task
1. Implement a `min_edit_distance` function using dynamic programming.
2. Show the DP matrix for the following test words: kitten → sitting.

In [ ]:
def min_edit_distance(word1: str, word2: str, insertion_cost=1, deletion_cost=1, substitution_cost=1, verbose=False):
    m, n = len(word1), len(word2)
    
    # Initializing DP matrix (size: (m+1) x (n+1))
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    
    # Base cases: transforming from empty string
    for i in range(m + 1):
        dp[i][0] = i * deletion_cost
    for j in range(n + 1):
        dp[0][j] = j * insertion_cost
    
    # Filling the DP matrix
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if word1[i - 1] == word2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]  # No cost if characters match
            else:
                dp[i][j] = min(
                    dp[i - 1][j] + deletion_cost,     # Deletion
                    dp[i][j - 1] + insertion_cost,    # Insertion
                    dp[i - 1][j - 1] + substitution_cost  # Substitution
                )
    
    # Printing DP matrix if verbose
    if verbose:
        print("DP Matrix:")
        print("    " + "  ".join([" "] + list(word2)))
        for i, row in enumerate(dp):
            if i == 0:
                prefix = " "
            else:
                prefix = word1[i-1]
            print(prefix, row)
    
    return dp[m][n]


In [3]:
distance = min_edit_distance("kitten", "sitting", verbose=False)
print("\nMinimum Edit Distance:", distance)



Minimum Edit Distance: 3


#### Reflective Question

**Why is minimum edit distance a good measure of similarity? When might it fail?**  
**Answer:**  
MED is a good measure for surface-level similarity as it directly measures how many changes are required to convert one string into another. This helpful in techniques such as spell checking, DNA sequence alignment etc. However, it is unable to capture deep semantic level similarity. For example, doog and puppy require 4 edits but are very similar semantically. I is also very sensitive to the word order. e-g. eat food and food eat has two edits but humans perceive them very similar. It doesn't have any account of the context as it looks at characters only and not grammar or meaning.  
Hence, MED is a good measure for surface level similarity but is not recommended for deep semantic level similarity due to above limitations.


## Part 2: Weighted Typo Mystery

In real life, not all typos are equal. Common mistakes, like swapping adjacent letters, should be "cheaper" than rare mistakes.

For example, on your keyboard:
- Substituting `q` ↔ `w` would cost 0.5, as they're next to each other

- Substituting `q` ↔ `b` would cost 2.0 as they're far away from each other

### Task
1. Extend your function to support a **custom cost matrix**.
    - Cost of 0.5 for adjacent characters and 2.0 for non-adjacent characters
2. Use it to compute the distance between words with weighted costs.
    - You have been given a helper function `are_adjacent(c1, c2)` which returns True if both c1 and c2 are next to each other on the keyboard, and False otherwise.
3. Compare results between standard MED and weighted MED for sample pairs.

In [ ]:
def are_adjacent(c1: str, c2: str) -> bool:
    # QWERTY keyboard layout as a 2D matrix
    keyboard = [
        ['q', 'w', 'e', 'r', 't', 'y', 'u', 'i', 'o', 'p'],
        ['a', 's', 'd', 'f', 'g', 'h', 'j', 'k', 'l'],
        ['z', 'x', 'c', 'v', 'b', 'n', 'm']
    ]

    # Function to find position of a character in the keyboard matrix
    def find_position(char):
        char = char.lower()
        for i, row in enumerate(keyboard):
            if char in row:
                return (i, row.index(char))
        return None

    pos1 = find_position(c1)
    pos2 = find_position(c2)

    if not pos1 or not pos2:
        return False  # character not on the keyboard

    r1, c1 = pos1
    r2, c2 = pos2

    # Checking adjacency: max distance of 1 in both row and column
    return abs(r1 - r2) <= 1 and abs(c1 - c2) <= 1 and pos1 != pos2

In [ ]:
def weighted_min_edit_distance(word1: str, word2: str, insertion_cost=1, deletion_cost=1, verbose=False):
    m, n = len(word1), len(word2)
    
    # Initializing DP matrix
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    
    # Base cases
    for i in range(m + 1):
        dp[i][0] = i * deletion_cost
    for j in range(n + 1):
        dp[0][j] = j * insertion_cost
    
    # Filling DP matrix
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if word1[i - 1] == word2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]  # No cost
            else:
                # Weighted substitution cost
                if are_adjacent(word1[i - 1], word2[j - 1]):
                    sub_cost = 0.5
                else:
                    sub_cost = 2.0
                dp[i][j] = min(
                    dp[i - 1][j] + deletion_cost,   # Deletion
                    dp[i][j - 1] + insertion_cost,  # Insertion
                    dp[i - 1][j - 1] + sub_cost     # Substitution
                )
    
    if verbose:
        print("Weighted DP Matrix:")
        print("    " + "  ".join([" "] + list(word2)))
        for i, row in enumerate(dp):
            if i == 0:
                prefix = " "
            else:
                prefix = word1[i-1]
            print(prefix, row)
    
    return dp[m][n]


In [15]:
# Standard MED
print("Standard MED (kitten → sitting):", min_edit_distance("kitten", "sitting"))

# Weighted MED
print("Weighted MED (kitten → sitting):", weighted_min_edit_distance("kitten", "sitting"))

# Close typo (q → w, adjacent keys)
print("Standard MED (q → w):", min_edit_distance("q", "w"))
print("Weighted MED (q → w):", weighted_min_edit_distance("q", "w"))

# Distant typo (q → b, far keys)
print("Standard MED (q → b):", min_edit_distance("q", "b"))
print("Weighted MED (q → b):", weighted_min_edit_distance("q", "b"))


Standard MED (kitten → sitting): 3
Weighted MED (kitten → sitting): 5
Standard MED (q → w): 1
Weighted MED (q → w): 0.5
Standard MED (q → b): 1
Weighted MED (q → b): 2


## Part 3: The Spell-Crime Case

Your detective agency just got a request: a manuscript full of errors.
The task: for each suspicious word, suggest the **top 3 candidate corrections** from a given dictionary, ranked by minimum edit distance.

### Task
1. Implement a function `suggest_corrections(word, dictionary, top_n=3)`.
2. Use your MED function (both standard and weighted) to generate corrections.
3. Analyze how the results differ depending on the cost function.

In [ ]:
def suggest_corrections(word, dictionary, weighted=False, top_n=3):
    results = []

    for candidate in dictionary:
        if weighted:
            dist = weighted_min_edit_distance(word, candidate)
        else:
            dist = min_edit_distance(word, candidate)
        results.append((candidate, dist))
    
    # Sorting by distance (smallest first), then alphabetically
    results.sort(key=lambda x: (x[1], x[0]))
    
    return results[:top_n]


dictionary = ["spelling", "selling", "spring", "smelling"]
print("Results for Standard MED:")
print(suggest_corrections("speling", dictionary, weighted=False))

print("\nResults for Weighted MED:")
print(suggest_corrections("speling", dictionary, weighted=True))

Results for Standard MED:
[('spelling', 1), ('selling', 2), ('smelling', 2)]

Results for Weighted MED:
[('spelling', 1), ('spring', 1.5), ('selling', 2)]


## Reflective Question  
**Does weighted MED reveal more insight on the similarity between two words than standard MED? If so, then explain how.**

Answer:
Yes, weighted MED provides more insight than standard MED because it accounts for how realistic certain errors are. For example, substituting adjacent keys (like t → y) costs less than distant ones (like t → c). This makes the measure more aligned with real typing mistakes, leading to more meaningful word similarity judgments.


## Part 4: Language Identification

A linguistic detective agency has received a single, typo-ridden word from a spy's message. They suspect the word reveals the spy's native language. Different languages have different common character sequences and typo patterns. Your task is to use MED to identify the likely language of origin.

You will identify a language by calculating the average MED of the typoed word against a corpus of known words for each language. The language with the lower average MED is the most likely candidate.

In [ ]:
english_corpus = ["apple", "banana", "cat", "dog", "house"]
german_corpus = ["apfel", "banane", "katze", "hund", "haus"]
typo_word = "dohg"

def identify_language(typo_word, english_corpus, german_corpus):
    # Helper to compute average MED
    def average_med(word, corpus):
        total = 0
        for w in corpus:
            total += min_edit_distance(word, w)  # reusing MED from Part 1
        return total / len(corpus)

    eng_score = average_med(typo_word, english_corpus)
    ger_score = average_med(typo_word, german_corpus)
    print(f"Average MED (English): {eng_score:.2f}")
    print(f"Average MED (German): {ger_score:.2f}")
    if eng_score < ger_score:
        return "English"
    elif ger_score < eng_score:
        return "German"
    else:
        return "Tie"
print("Likely Language:", identify_language(typo_word, english_corpus, german_corpus))


Average MED (English): 4.00
Average MED (German): 4.80
Likely Language: English
